<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/Data_Ac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing python libraries that will be use to get data from yahoo finance

In [1]:
! pip install yahooquery
! pip install urllib3==1.26
! pip install requests==2.25
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Importing modules that will be used to filter data

In [2]:
from yahooquery import Ticker
from bs4 import BeautifulSoup

import concurrent.futures
import pandas as pd
import requests
import re
import os

Scrapping Data from Infomoney website, getting a list of stocks from IBOVESPA.

In [3]:
url = requests.get('https://www.infomoney.com.br/cotacoes/empresas-b3/')

soup = BeautifulSoup(url.text, 'html.parser')

ticker_name_list = soup.find_all('td', class_ = 'strong')

ticker = str(ticker_name_list)

tickers = re.findall('>.*</',ticker)

tkrs = []

# strnot = ("11","F","34")
for i in tickers:
  if "11" not in i and "F" not in i:
    tkr = i.strip("</>")
    tkr = tkr + ".SA"
    tkrs.append(tkr)

In [4]:
def fetch_historical_financial(tickers):
    # Yahoo finance requisition getting companies ticker
    sumary = Ticker(tickers)

    # getting companies financial data
    data = sumary.all_financial_data()

    tickercol = data.index.tolist()

    df = pd.DataFrame(data)

    df['Ticker'] = tickercol

    return df

In [5]:
def fetch_historical_balance(tickers):
    # Balance Ticker lists
    balance = Ticker(tickers)

    # Getting companies balance data
    data = balance.balance_sheet(trailing=False)

    tickers = data.index.tolist()

    df = pd.DataFrame(data)

    df['Ticker'] = tickers  # Add 'Ticker' column to the DataFrame
    # df = df.reset_index(drop=True)  # Reset the index of the final DataFrame
    # df['Symbol'] = tickers

    return df

In [6]:
def fetch_historical_finandata(tickers):
    
    # Balance Ticker lists
    financial = Ticker(tickers)

    # Getting companies balance data
    types = ['symbol', 'TotalDebt', 'TotalAssets', 'EBIT', 'EBITDA', 'PeRatio']
        
    data = financial.get_financial_data(types, trailing=False)

    tickers = data.index.tolist()

    df = pd.DataFrame(data)

    df['Ticker'] = tickers  # Add 'Ticker' column to the DataFrame

    df = df.reset_index(drop=True)

    return df

In [7]:
def fetch_historical_price(tickers):
    
    pricetkrs = Ticker(tickers)

    dataprice = pricetkrs.price

    return dataprice

In [8]:
financial = "AAPL"

fetch_historical_price(financial)

{'AAPL': 'Invalid Cookie'}

In [53]:
# Define a function to fetch data for each batch of tickers
def fetch_data_batch(tkrs,FS):
    if FS == 1:
        data = fetch_historical_financial(tkrs)

    elif FS == 2:
        data = fetch_historical_balance(tkrs)

    elif FS == 3:
        data = fetch_historical_finandata(tkrs)

    elif FS == 4:
        data = fetch_historical_price(tkrs)

    return data

In [54]:
def ElementsList(elements_per_list,tkrs):
    # Divide the ticker list into smaller lists
    smaller_lists = [tkrs[i:i+elements_per_list] for i in range(0, len(tkrs), elements_per_list)]
    
    return smaller_lists

In [55]:
# # # Function that processes operations based on different flags
# # # BS: Batch size
# # # FS: Function Signal
# # # TL: Ticker List

def Process(BS,TL,FS):
    # Lists of elements
    lists = ElementsList(BS,TL)
    
    # dataframe that contains the results of the processed files
    df_final = pd.DataFrame()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit the threads for processing
        futures = [executor.submit(fetch_data_batch, list_batch, FS) for list_batch in lists[:BS]]

        # Iterate through the completed threads and concatenate the results
        for future in concurrent.futures.as_completed(futures):
            data = future.result()
            df_final = pd.concat([df_final, data])

    # Reset the index of the final DataFrame
    df_final = df_final.reset_index(drop=True)

    return df_final

In [56]:
historical_financial = Process(20,tkrs,1)

In [57]:
historical_balance = Process(20,tkrs,2)

In [58]:
historical_finandata = Process(20,tkrs,3)

In [59]:
historical_data = Process(20,tkrs,4)

In [60]:
# Define the output directory and filename
output_directory = "../RAW_DATA/tst"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Files that will be created
# "AtivosBR.xlsx"
filenames = ["Beta.xlsx","Financial.xlsx","Price.xlsx"]

# Dataframes of objects
# historical_data
dataframes = [historical_financial,historical_balance,historical_finandata]

# Sheet names
# "historical_data",
sheet_names = ["historical_financial","historical_balance","historical_finance_data"]

for item in range(0,len(filenames)):
    # Specify the full file path
    output_file = os.path.join(output_directory, filenames[item])
    #
    dataframes[item].to_excel(excel_writer=output_file, sheet_name=sheet_names[item], engine='openpyxl')